**Description** : Clean Mock Train Data

# Import Libraries

In [1]:
import pandas as pd

# Explore Data

In [29]:
df = pd.read_csv("D:\\JT\GTU OMS\\Data and Visual Analytics (CSE 6242)\\Project\\data\\raw\\1000rows_translated_german.csv")[['text', 'label']]
df.head()

,text,label
0,Garrard Conley is an assistant professor of cr...,2
1,General Services Administration GSA Administra...,2
2,"MADRID AP More than 17,000 fans were at the Ca...",0
3,"The weekend isn’t over yet, so that means ther...",0
4,Dear Prudence is online weekly to chat live wi...,0


In [32]:
# df.info()
df[df['label'].isin(['0','1'])].shape

(577, 2)

In [24]:
# Remove rows with missing text values
df = df[df['text'].notna()]
df.shape

(20761, 5)

In [9]:
data = pd.read_csv("D:\\JT\GTU OMS\\Data and Visual Analytics (CSE 6242)\\Project\\src\\modelling\\train_results.csv")
a = data.mean()
b = data.max()

pd.concat([pd.DataFrame(a).transpose(), pd.DataFrame(b).transpose()])


,fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted,test_roc_auc,model
0,16.8024,0.379542,0.941795,0.942481,0.941795,0.941765,0.98681,NaN
0,51.4022,1.2448,0.963877,0.963938,0.963877,0.963872,0.993861,RandomForest


In [28]:
from textblob import TextBlob
import pandas as pd

pos_dic = {
            'noun' : ['NN','NNS','NNP','NNPS'],
            'pron' : ['PRP','PRP$','WP','WP$'],
            'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
            'adj' :  ['JJ','JJR','JJS'],
            'adv' : ['RB','RBR','RBS','WRB'],
            'prep' : ['IN'],    # Preposition
            'int': ['UH'],    # Interjections
            'dt': ['DT'],    # Determiners
            'cc': ['CC', 'CD'],    # Conjunction
            'md': ['MD']    # Modal
        }

wiki = TextBlob('My name is Joseph. I am 28. I like Data.')
tags = [x[1] for x in wiki.tags]
tag_count = pd.Series(tags).value_counts().to_dict()

cnt = {
'noun': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['noun']]), 
'pron': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['pron']]), 
'verb': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['verb']]), 
'adj': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['adj']]), 
'adv': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['adv']]),
'prep': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['prep']]),
'int': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['int']]),
'dt': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['dt']]),
'cc': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['cc']]),
'md': sum([tag_count[k] for k in tag_count.keys() if k in pos_dic['md']]),
}

density = {k: v/sum(tag_count.values())*100 for k, v in cnt.items()}

# index  of  formality: F = (noun + adjective + preposition – pronoun – verbs – participles – adverbs – interjections + 100)/2
iof = (cnt['noun'] + cnt['adj'] + cnt['prep'] - cnt['pron'] - cnt['verb'] - cnt['adv'] - cnt['int'])/2

# index of the lexical density: ratio  of  function  words  to  content words
iol = (cnt['pron']+cnt['prep']+cnt['int']+cnt['dt']+cnt['cc']+cnt['md'])/(cnt['noun']+cnt['verb']+cnt['adj']+cnt['adv'])

# POS diversity
pos_diversity = len(set(tags))/len(tags)

print(density, iof, iol, pos_diversity)

{'noun': 30.0, 'pron': 30.0, 'verb': 30.0, 'adj': 0.0, 'adv': 0.0, 'prep': 0.0, 'int': 0.0, 'dt': 0.0, 'cc': 10.0, 'md': 0.0} -1.5 0.6666666666666666 0.7


In [22]:
d = ({'a':1, 'b':2}, 3, 4)

e,f,g = d

sum(e.values())

3

In [4]:
pd.DataFrame({'names':['a','b'], 'values':[1, 0]})

,names,values
0,a,1
1,b,0


In [16]:
[ "Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim."]*4 

['Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.',
 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.',
 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.',
 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.']

In [17]:
import pandas as pd

pd.DataFrame(
    {'Model': ['Model 1', 'Model 2', "Model 3", 'Model 4'],
    'Results': [ "Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim."]*4}
).to_dict('records')

[{'Model': 'Model 1',
  'Results': 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.'},
 {'Model': 'Model 2',
  'Results': 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.'},
 {'Model': 'Model 3',
  'Results': 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.'},
 {'Model': 'Model 4',
  'Results': 'Lorem ipsum dolor sit amet, tollit discere inermis pri ut. Eos ea iusto timeam, an prima laboramus vim.'}]

# Modelling

In [1]:
import pandas as pd
from pycaret.classification import *

In [2]:
# import data
data = pd.read_csv('D:\JT\GTU OMS\Data and Visual Analytics (CSE 6242)\Project\src\processing\data_features.csv').drop(['text','Unnamed: 0'], axis=1)
data = data.dropna(subset=['label'])
data['label'] = data['label'].astype(str)

## Setting up Environment in PyCaret

In [5]:
exp_clf102 = setup(data = data, target = 'label', session_id=123,
                  numeric_features=['polarity_', 'subjectivity_'],
                  normalize = True, 
                  ignore_low_variance = True,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95)

,Description,Value
0,session_id,123
1,Target,label
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 2.0: 1"
4,Original Data,"(4917, 23)"
5,Missing Values,True
6,Numeric Features,22
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


## Comparing All Models

In [6]:
top3 = compare_models(n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.5740,0.6053,0.5729,0.5737,0.5728,0.1459,0.1462,5.2820
gbc,Gradient Boosting Classifier,0.5731,0.6174,0.5723,0.5733,0.5720,0.1447,0.1452,0.2450
lr,Logistic Regression,0.5655,0.5885,0.5649,0.5657,0.5649,0.1300,0.1302,0.6640
et,Extra Trees Classifier,0.5652,0.5946,0.5643,0.5650,0.5647,0.1288,0.1290,0.1730
lightgbm,Light Gradient Boosting Machine,0.5652,0.5906,0.5643,0.5650,0.5644,0.1287,0.1289,0.6570
rf,Random Forest Classifier,0.5650,0.5958,0.5645,0.5652,0.5644,0.1290,0.1292,0.2470
ada,Ada Boost Classifier,0.5644,0.5959,0.5636,0.5643,0.5638,0.1272,0.1275,0.0710
ridge,Ridge Classifier,0.5565,0.0000,0.5545,0.5561,0.5543,0.1094,0.1104,0.0070
lda,Linear Discriminant Analysis,0.5565,0.5793,0.5545,0.5561,0.5543,0.1094,0.1104,0.0080
xgboost,Extreme Gradient Boosting,0.5455,0.5804,0.5445,0.5451,0.5448,0.0891,0.0893,0.5660


## Create a Model

In [ ]:
dt = create_model('dt', fold = 5)

## Tune Model

In [ ]:
tuned_rf = tune_model(rf, optimize = 'AUC')

In [ ]:
# Plot model calibration
plot_model(tuned_rf, plot='calibration')

## Bagging

In [ ]:
bagged_dt = ensemble_model(dt)

## Boosting

In [ ]:
boosted_dt = ensemble_model(dt, method = 'Boosting')

## Blending

In [ ]:
# train individual models to blend
lightgbm = create_model('lightgbm', verbose = False)
dt = create_model('dt', verbose = False)
lr = create_model('lr', verbose = False)



# blend individual models
blend_soft = blend_models(estimator_list = [lightgbm, dt, lr], method = 'soft')


## Stacking

In [ ]:
stack_soft = stack_models(top3)